## Multi-Label Emotion Classification Using NLP: Analyzing Emotional Tone in Social Media and Text Data

Project overview



Business Understanding

Data Understanding 

## Importing Necessary Libraries

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import json
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from langdetect import detect
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

In [40]:
# Load the training data
train_data = pd.read_csv('full_dataset/train.tsv', sep='\t', header=None ,names=['text', 'label', 'id'])

# Load the validation data
dev_data = pd.read_csv('full_dataset/dev.tsv', sep='\t', header=None,names=['text', 'label', 'id'])
# test data 
test_data = pd.read_csv('full_dataset/test.tsv', sep='\t', header=None,names=['text', 'label', 'id'])

#Initial Data inspection
train_data.head()
dev_data.head()
test_data.head()


,text,label,id
0,I’m really sorry about your situation :( Altho...,25,eecwqtt
1,It's wonderful because it's awful. At not with.,0,ed5f85d
2,"Kings fan here, good luck to you guys! Will be...",13,een27c3
3,"I didn't know that, thank you for teaching me ...",15,eelgwd1
4,They got bored from haunting earth for thousan...,27,eem5uti


In [51]:
#splitting labels and counting class Lengths
train_data['List of classes'] = train_data['label'].apply(lambda x: x.split(','))
train_data['Len of classes'] = train_data['List of classes'].apply(lambda x: len(x))
dev_data['List of classes'] = dev_data['label'].apply(lambda x: x.split(','))
dev_data['Len of classes'] = dev_data['List of classes'].apply(lambda x: len(x))

This block reads a JSON file (ekman_mapping.json) that contains mappings from finer-grained emotion labels to broader emotion categories (such as "anger", "joy", etc.).

In [45]:
with open('full_dataset/ekman_mapping.json') as file:
    ekman_mapping = json.load(file)

The JSON file is loaded into the variable ekman_mapping, which can then be used to map specific emotions to broader emotion categories later on.

The Next line of code opens a text file (emotions.txt) that contains a list of emotions (one emotion per line).
The file is read into a string, which is then split into a list of emotions using split("\n"), where each emotion is stored on a new line.

In [61]:
emotion_file = open("full_dataset/emotions.txt", "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")
print(emotion_list)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [48]:
def idx2class(idx_list):
    arr = []
    for i in idx_list:
        arr.append(emotion_list[int(i)])
    return arr

Using the emotion_list (which was previously loaded), this function (idx2class) takes a list of emotion label indices (idx_list) and converts them to their corresponding emotion names.
After being translated to integers, the emotion label indices are mapped to the corresponding emotion class names.

In [53]:
#Applying Emotion Mapping to DataFrames.
train_data['Emotions'] = train_data['List of classes'].apply(idx2class)
dev_data['Emotions'] = dev_data['List of classes'].apply(idx2class)

In [54]:
def EmotionMapping(emotion_list):
    map_list = []
    
    for i in emotion_list:
        if i in ekman_mapping['anger']:
            map_list.append('anger')
        if i in ekman_mapping['disgust']:
            map_list.append('disgust')
        if i in ekman_mapping['fear']:
            map_list.append('fear')
        if i in ekman_mapping['joy']:
            map_list.append('joy')
        if i in ekman_mapping['sadness']:
            map_list.append('sadness')
        if i in ekman_mapping['surprise']:
            map_list.append('surprise')
        if i == 'neutral':
            map_list.append('neutral')
            
    return map_list

In [55]:
train_data['Mapped Emotions'] = train_data['Emotions'].apply(EmotionMapping)
dev_data['Mapped Emotions'] = dev_data['Emotions'].apply(EmotionMapping)

In [56]:
train_data['anger'] = np.zeros((len(train_data),1))
train_data['disgust'] = np.zeros((len(train_data),1))
train_data['fear'] = np.zeros((len(train_data),1))
train_data['joy'] = np.zeros((len(train_data),1))
train_data['sadness'] = np.zeros((len(train_data),1))
train_data['surprise'] = np.zeros((len(train_data),1))
train_data['neutral'] = np.zeros((len(train_data),1))

dev_data['anger'] = np.zeros((len(dev_data),1))
dev_data['disgust'] = np.zeros((len(dev_data),1))
dev_data['fear'] = np.zeros((len(dev_data),1))
dev_data['joy'] = np.zeros((len(dev_data),1))
dev_data['sadness'] = np.zeros((len(dev_data),1))
dev_data['surprise'] = np.zeros((len(dev_data),1))
dev_data['neutral'] = np.zeros((len(dev_data),1))

In [57]:
for i in ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise','neutral']:
    train_data[i] = train_data['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)
    dev_data[i] = dev_data['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)
    

In [58]:
train_data.head()

,text,label,id,List of classes,Len of classes,Emotions,Mapped Emotions,anger,disgust,fear,joy,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],1,0,0,0,0,0,0
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,1,0,0,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[anger],1,0,0,0,0,0,0


In [59]:
dev_data.head()

,text,label,id,List of classes,Len of classes,Emotions,Mapped Emotions,anger,disgust,fear,joy,sadness,surprise,neutral
0,Is this in New Orleans?? I really feel like th...,27,edgurhb,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
1,"You know the answer man, you are programmed to...","4,27",ee84bjg,"[4, 27]",2,"[approval, neutral]","[joy, neutral]",0,0,0,1,0,0,1
2,I've never been this sad in my life!,25,edcu99z,[25],1,[sadness],[sadness],0,0,0,0,1,0,0
3,The economy is heavily controlled and subsidiz...,"4,27",edc32e2,"[4, 27]",2,"[approval, neutral]","[joy, neutral]",0,0,0,1,0,0,1
4,He could have easily taken a real camera from ...,20,eepig6r,[20],1,[optimism],[joy],0,0,0,1,0,0,0
